# 多元線性回歸 — CRISP-DM 全流程

> **資料集**：Kaggle Wine Quality（11 個理化特徵）

本 Notebook 依照 **CRISP-DM** 流程：Business Understanding → Data Understanding → Data Preparation → Modeling → Evaluation → Deployment。

## Business Understanding
- 目標：根據葡萄酒理化特徵預測 **品質分數 (quality)**，作為品控或配方參考。
- 成功指標：RMSE 降低、R² 提升，並提供預測區間做風險溝通。

## Data Understanding
- 來源：Kaggle Wine Quality。
- 特徵數：11（固定酸度、揮發性酸、檸檬酸、殘糖、氯化物、自由/總二氧化硫、密度、pH、硫酸鹽、酒精）。
- 目標：`quality`（0–10 分）。

In [ ]:
# Imports
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
import scipy.stats as stats

STUDENT_ID = "7114056076"  
DATA_PATH = "WineQT.csv"  
TARGET = "quality"
RANDOM_STATE = 42
TEST_SIZE = 0.2
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

### 載入與檢視資料

In [ ]:
df = pd.read_csv(DATA_PATH)
display(df.head())
df.shape

### 缺失值、型態與描述統計

In [ ]:
display(df.info())
display(df.describe())
df.isna().sum()

## Data Preparation
- 去重與缺失值處理
- 僅保留數值欄位
- Train/Test 切分

In [ ]:
df = df.drop_duplicates().dropna().reset_index(drop=True)
if 'type' in df.columns:
    df['type'] = (df['type'].astype(str).str.lower().str.contains('white')).astype(int)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
features = [c for c in num_cols if c != TARGET]
X, y = df[features], df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

## Modeling — Baseline 與特徵選擇

In [ ]:
baseline = Pipeline([('scaler', StandardScaler()), ('lr', LinearRegression())])
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
import numpy as np
rmse_cv = -np.mean(cross_val_score(baseline, X, y, scoring='neg_root_mean_squared_error', cv=cv))
rmse_cv

In [ ]:
lasso = Pipeline([('scaler', StandardScaler()), ('lasso', LassoCV(cv=5, random_state=RANDOM_STATE, n_alphas=50))])
lasso.fit(X_train, y_train)
coefs = lasso.named_steps['lasso'].coef_
selected = [f for f, c in zip(features, coefs) if abs(c) > 1e-8]
if not selected:
    corr = df[features + [TARGET]].corr()[TARGET].abs().drop(TARGET).sort_values(ascending=False)
    selected = corr.head(min(6, len(features))).index.tolist()
selected

### 以 StatsModels OLS 擬合（用於信賴/預測區間）

In [ ]:
X_train_sel = sm.add_constant(X_train[selected], has_constant='add')
X_test_sel  = sm.add_constant(X_test[selected], has_constant='add')
ols = sm.OLS(y_train, X_train_sel).fit()
print(ols.summary().as_text())

## Evaluation

In [ ]:
pred_test = ols.predict(X_test_sel)
r2 = r2_score(y_test, pred_test)
mae = mean_absolute_error(y_test, pred_test)
rmse = mean_squared_error(y_test, pred_test, squared=False)
r2, mae, rmse

### 預測圖（含 95% 預測區間）

In [ ]:
pred_res = ols.get_prediction(X_test_sel).summary_frame(alpha=0.05)
plot_df = pd.DataFrame({
    'actual': y_test.values,
    'pred': pred_res['mean'].values,
    'pi_low': pred_res['obs_ci_lower'].values,
    'pi_high': pred_res['obs_ci_upper'].values
}).sort_values('pred').reset_index(drop=True)

plt.figure(figsize=(9,5))
plt.plot(plot_df.index, plot_df['pred'], label='Predicted')
plt.fill_between(plot_df.index, plot_df['pi_low'], plot_df['pi_high'], alpha=0.2, label='95% PI')
plt.scatter(plot_df.index, plot_df['actual'], s=18, alpha=0.7, label='Actual')
plt.title('Predictions with 95% Prediction Interval (Test Set)')
plt.xlabel('Sorted Test Samples'); plt.ylabel('quality'); plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'predictions_with_PI.png'), dpi=150)


### 殘差診斷

In [ ]:
residuals = y_test.values - pred_res['mean'].values
plt.figure(figsize=(8,4.8))
plt.scatter(pred_res['mean'].values, residuals, s=18, alpha=0.7); plt.axhline(0, lw=1)
plt.title('Residuals vs Fitted'); plt.xlabel('Fitted'); plt.ylabel('Residuals'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'residuals_vs_fitted.png'), dpi=150)

plt.figure(figsize=(8,4.8))
import scipy.stats as stats
stats.probplot(residuals, dist='norm', plot=plt); plt.title('Residual Q-Q Plot'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'residuals_qq.png'), dpi=150)


### 單變數線性回歸（可選）

In [ ]:
one_feat = 'alcohol' if 'alcohol' in features else selected[0]
X1_train = sm.add_constant(X_train[[one_feat]])
X1_test  = sm.add_constant(X_test[[one_feat]])
ols1 = sm.OLS(y_train, X1_train).fit()
pred1 = ols1.get_prediction(X1_test).summary_frame(alpha=0.05)

order = np.argsort(X_test[one_feat].values)
x_sorted = X_test[one_feat].values[order]
mean_sorted = pred1['mean'].values[order]
l_sorted = pred1['mean_ci_lower'].values[order]
u_sorted = pred1['mean_ci_upper'].values[order]

plt.figure(figsize=(8,4.8))
plt.plot(x_sorted, mean_sorted, label='Fitted line')
plt.fill_between(x_sorted, l_sorted, u_sorted, alpha=0.2, label='95% CI (mean)')
plt.scatter(X_test[one_feat], y_test, s=18, alpha=0.7, label='Actual')
plt.title(f'Simple Linear Regression: {one_feat} → {TARGET}')
plt.xlabel(one_feat); plt.ylabel(TARGET); plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, f'SLR_{one_feat}_with_CI.png'), dpi=150)
